PARTE 1: Abrir a página do pregão que contém as informações básicas e os links para cada item

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
import datetime
import time
import pandas as pd
import csv
import os

In [46]:
# URL da página do pregão
url = "https://contratos.sistema.gov.br/transparencia/compras?lei=LEI14133&lei_text=LEI14133&unidade_origem_id=9964&unidade_origem_id_text=160224+-+PARQUE+REGIONAL+DE+MANUTENCAO%2F5&modalidade_id=76&modalidade_id_text=05+-+Preg%C3%A3o"

# Obtendo a data atual
data_atual = datetime.date.today()

def iniciar():
    if os.path.exists('url_pregoes.csv'):
        try:
            url_pregoes_antiga = carregar_lista_pregoes()
        except:
            print(f'Erro ao carregar lista de pregões: {e}.')

        try:
                        
            url_pregoes_nova = baixar_nova_lista_pregoes(url)
        except:
            print(f'Erro ao baixar nova lista de pregões: {e}.')

        try:
                        
            df_itens = verificar_se_ha_novos_pregoes(url_pregoes_antiga, url_pregoes_nova)
        except:
            print(f'Erro na função verificar_se_ha_novos_pregoes: {e}.')

        try:
            
            salvar_dados(df_itens)

        except:
            
            print(f'Erro ao salvar os dados: {e}.')
            
            
    else:
        print('Não há lista de pregões salvas nesse computador.')
        url_pregoes_antiga = []
        
        #Baixando lista de pregões
        print('Obtendo a url dos pregões...')
        url_pregoes_nova = baixar_nova_lista_pregoes(url)
        
        print('Obtendo a url dos itens...')      
        url_itens = obter_url_dos_itens(url_pregoes_nova)
        
        print('URL dos itens obtida com sucesso!') 
        
        print('Obtendo os dados dos itens...')
        df_itens = atualizar_todos_os_dados(url_itens)
        
        print('Dados dos itens obtidos com sucesso!')
    
        salvar_dados(df_itens)
        
        return 'Dados Salvos com sucesso!'

In [47]:
def carregar_lista_pregoes():
    nome_arquivo = 'url_pregoes.csv'
    url_pregoes_antiga = []
    
    with open(nome_arquivo, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Pular o cabeçalho
        url_pregoes_antiga = [row[0] for row in reader]
    
    print('Lista de Pregões salvas no PC encontrada!')
        
    return url_pregoes_antiga

def baixar_nova_lista_pregoes(url):
    
    print('Baixando nova lista de pregões ...')
    # Configuração do WebDriver
    #options = webdriver.ChromeOptions()
    #options.add_argument('--headless')  # Executar em modo headless para não abrir uma janela do navegador
    #driver = webdriver.Chrome(options=options)
    driver = webdriver.Chrome()
    driver.get(url)

    # Selecionar a opção "Todos"
    select = Select(driver.find_element(By.NAME, 'crudTable_length'))
    select.select_by_value('-1')
    time.sleep(5)  # Esperar alguns segundos para garantir que a página carregue completamente

    # Obter e parsear o HTML da página
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    specific_url_part = 'https://contratos.sistema.gov.br/transparencia/compras/'
    url_pregoes_nova = [a_tag['href'].replace('show', 'itens') for a_tag in soup.find_all('a', href=True) if specific_url_part in a_tag['href']]
    


    # Gravar a lista em um arquivo CSV
    nome_arquivo = 'url_pregoes.csv'
    with open(nome_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f'Atualizado em {data_atual}'])  # Escrever a data da atualização
        writer.writerows([[url] for url in url_pregoes_nova])
        print('Nova Lista de pregões salva com sucesso!')
    
    return url_pregoes_nova

In [48]:
def verificar_se_ha_novos_pregoes(url_pregoes_antiga, url_pregoes_nova):
    
    novos_pregoes = list(set(url_pregoes_nova) - set(url_pregoes_antiga))
    
    if novos_pregoes == []:
        
        print('Não há novos pregões. Iniciando atualização dos saldos...')
        
        #Carregando df_itens
    
        print('Carregando DataFrame com os dados dos itens...')

        df_itens = pd.read_csv('df_itens_gerenciadora.csv', sep=';')
        
        #Para obter as informações de cada item

        #chamar função de atualizar o saldo dos pregões existentes
    
        df_itens = atualizar_saldo(df_itens)
        
        return df_itens
        
        
    else:
        
        print(f'Há novos pregões não cadastrados! \nObtendo os dados dos pregões: \n{novos_pregoes}')
                
        url_itens_novos = obter_url_dos_itens(novos_pregoes)

        df_itens_novos = atualizar_todos_os_dados(url_itens_novos)

  
        print('Dados dos novos pregões atualizados!')        
        
        #colocar um try encontrar df_itens salvos

        try:

            print('Tentando carregar dados dos pregões anteriores...')       
            
            df_itens_antigos = pd.read_csv('df_itens_gerenciadora.csv', sep = ';')
            
            print('Dados carregados com sucesso! Iniciando atualização dos saldos...')
            
            df_itens_antigos = atualizar_saldo(df_itens_antigos)
            
            print('Saldo dos pregões anteriores atualizado. Concatenando a tabela dos itens antigos com os novos...')
            
            df_itens = pd.concat([df_itens_novos, df_itens_antigos])
        
            return df_itens

        except:

            print('Não foi possível carregar os dados dos pregões anteriores.')

            df_itens = df_itens_novos

            return df_itens

In [49]:
def obter_unidade_objeto(df_itens_novos):
    
    #Obtendo as novas URL dos pregões

    lista_pregao_site_antigo = []
    
    for i in range(len(df_itens_novos)):
      url = "http://comprasnet.gov.br/ConsultaLicitacoes/download/download_editais_detalhe.asp?coduasg=" + str(df_itens_novos['UASG'][i])+"&modprp=5&numprp=" + str(df_itens_novos['Número do Pregão'][i]) + str(df_itens_novos['Ano do Pregão'][i])
      lista_pregao_site_antigo.append(url)
    
    lista_pregao_site_antigo = list(dict.fromkeys(lista_pregao_site_antigo))

    #obter os dados do pregão na primeira página

    linhas = []
    
    for url in lista_pregao_site_antigo:
    
        #Abrindo a página desejada
    
        driver = webdriver.Chrome()
        driver.get(url)
        #options = webdriver.ChromeOptions()
        #options.add_argument('--headless')  # Executar em modo headless para não abrir uma janela do navegador
        time.sleep(5)  # Aguarde a página carregar
    
        for i in range(len(driver.find_elements(By.XPATH, "//span[@class='tex3b']")[2:-1])):
            
            numeropregao_ano = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[0].text.split('Nº')[1].strip()
            objeto = driver.find_element(By.XPATH, "//p[contains(text(),'Objeto:')]").text.split('\n')[1]
            numero_item = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[i+2].text.split('-')[0].strip()
            descricao_item = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[i+2].text.split('-')[1].strip()
            descricao_complementar = driver.find_elements(By.XPATH, "//span[@class='tex3']")[i].text.split('\n')[0]
            unidade_medida = driver.find_elements(By.XPATH, "//span[@class='tex3']")[i].text.split('\n')[5].split(':')[1].strip()
        
            dados = [numeropregao_ano, objeto, numero_item, descricao_item, descricao_complementar, unidade_medida]
            
            linhas.append(dados)
    
        
        #verificar se tem próxima página para obter os dados
        
        while True:
            try:
                # Tente encontrar o botão "Próxima" e clique nele
                next_button = driver.find_element(By.ID, "proximas").click()
                
                time.sleep(5)  # Aguarde a página carregar
                
                for i in range(len(driver.find_elements(By.XPATH, "//span[@class='tex3b']")[2:-1])):
                    
                    numeropregao_ano = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[0].text.split('Nº')[1].strip()
                    objeto = driver.find_element(By.XPATH, "//p[contains(text(),'Objeto:')]").text.split('\n')[1]
                    numero_item = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[i+2].text.split('-')[0].strip()
                    descricao_item = driver.find_elements(By.XPATH, "//span[@class='tex3b']")[i+2].text.split('-')[1].strip()
                    descricao_complementar = driver.find_elements(By.XPATH, "//span[@class='tex3']")[i].text.split('\n')[0]
                    unidade_medida = driver.find_elements(By.XPATH, "//span[@class='tex3']")[i].text.split('\n')[5].split(':')[1].strip()
                
                    dados = [numeropregao_ano, objeto, numero_item, descricao_item, descricao_complementar, unidade_medida]
                    
                    linhas.append(dados)
                    
            except:
                # Fechar o WebDriver
                driver.quit()
                # Se não encontrar o botão "Próxima", saia do loop
                break



    #Cabeçalho
    headers = ["Número da Compra","Objeto","Número do Item","Descrição", "Descrição Complementar","Unidade de Fornecimento"]
    
    
    #transformando os dados em dataframe
    df_itens_novos_site_antigo = pd.DataFrame(linhas, columns = headers)

    #apagar
    print('imprimir df_itens_novos_site_antigo')
    display(df_itens_novos_site_antigo)
    
    #obtendo o número do pregão e o ano
    
    ano_pregao = []
    numero_pregao = []
    
    for i in range(len(df_itens_novos_site_antigo['Número da Compra'])):
        
        numero = df_itens_novos_site_antigo['Número da Compra'][i].split('/')[0]
        numero_pregao.append(numero)
        
        ano = df_itens_novos_site_antigo['Número da Compra'][i].split('/')[1]
        ano_pregao.append(ano)
    
    df_itens_novos_site_antigo['Ano do Pregão'] = ano_pregao
    df_itens_novos_site_antigo['Número do Pregão'] = numero_pregao
    
    df_itens_novos_site_antigo['Ano do Pregão'] = df_itens_novos_site_antigo['Ano do Pregão'].astype(int)
    df_itens_novos_site_antigo['Ano do Pregão'] = df_itens_novos_site_antigo['Ano do Pregão'].astype('string')
    
    df_itens_novos_site_antigo['Número do Pregão'] = df_itens_novos_site_antigo['Número do Pregão'].astype(int)
    df_itens_novos_site_antigo['Número do Pregão'] = df_itens_novos_site_antigo['Número do Pregão'].astype('string')

    df_itens_novos_site_antigo['Número do Item'] = df_itens_novos_site_antigo['Número do Item'].astype(int)
    df_itens_novos_site_antigo['Número do Item'] = df_itens_novos_site_antigo['Número do Item'].astype('string')

    return df_itens_novos_site_antigo

In [50]:
def salvar_dados(df_itens):
    
    #Salvando o df em arquivo csv
    nome_arquivo = "df_itens_gerenciadora.csv"

    # Exportando o DataFrame para um arquivo CSV com delimitador ';' e codificação UTF-8
    df_itens.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)

    print('Arquivo com a tabela dos itens salva com sucesso!')
    
    #Gravar a lista em um arquivo CSV
    
    df_itens['Link do Item'].to_csv('url_itens.csv', sep=';', encoding='utf-8', index=False)
    
    print('Lista com a url dos itens salva com sucesso!')

In [51]:
def obter_url_dos_itens(novos_pregoes):
    
    url_itens = []

    for url in novos_pregoes:       

    # Configuração do WebDriver
        #options = webdriver.ChromeOptions()
        #options.add_argument('--headless')  # Executar em modo headless para não abrir uma janela do navegador
        #driver = webdriver.Chrome(options=options)
        driver = webdriver.Chrome()
        driver.get(url)

        #apagar
        print(url)

        # Encontrar o elemento select pelo ID
        select_element = driver.find_element(By.NAME, 'crudTable_length')

        # Criar um objeto Select
        select = Select(select_element)

        # Selecionar a opção "Todos"
        select.select_by_value('-1')

        # Esperar alguns segundos para garantir que a página carregue completamente
        time.sleep(15)

        # Obter o HTML da página
        page_source = driver.page_source

        # Fechar o WebDriver
        driver.quit()

        # Parsear o HTML com Beautiful Soup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontrar os links para cada item
        specific_url_part = 'https://contratos.sistema.gov.br/transparencia/compras/'

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if specific_url_part in href:
                url_itens.append(href)    

    url_itens = [url for url in url_itens if 'show' in url]

    #apagar
    print(url_itens)

    # Gravar a lista em um arquivo CSV
    nome_arquivo = 'url_itens.csv'
    with open(nome_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f'Atualizado em {data_atual}'])  # Escrever a data da atualização
        writer.writerows([[url] for url in url_itens])
        print('Nova Lista de url dos itens salva com sucesso!')
    
        
    return url_itens

In [52]:
def atualizar_todos_os_dados(url_itens):

    linhas = []

    for url in url_itens:
        
        # Enviar uma solicitação GET para obter o conteúdo da página
        response = requests.get(url)
        
        # Verificar se a solicitação foi bem-sucedida

        if response.status_code == 200:              

            # Parsear o HTML da página com Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            #Pregão
            pregao = soup.find_all('div', class_='header-title')[1].text.strip().replace('Itens da compra: 160224 - ','')

            # Encontrar todas as tabelas na página
            tables = soup.find_all('table')
            #Define a nova linha
            linha = [
                pregao, #Pregão
                tables[0].find_all('span')[0].text.strip(), #Item
                tables[0].find_all('span')[2].text.strip(), #Descrição
                tables[0].find_all('span')[3].text.strip(), #Descrição detalhada
                #tables[0].find_all('span')[4].text.strip(), #Qtd. Total
                tables[0].find_all('span')[5].text.strip(), #Vig. Início ARP
                tables[0].find_all('span')[6].text.strip(), #Vig. Fim ARP
                tables[1].find_all('td')[0].text.strip(), #Unidade
                #tables[1].find_all('td')[1].text.strip(), #Tipo UASG
                tables[1].find_all('td')[2].text.strip(), #Qtd. Autorizada

                tables[2].find_all('td')[0].text.strip(), #Fornecedor
                #tables[2].find_all('td')[1].text.strip(), #Qtd. Homologada
                tables[2].find_all('td')[2].text.strip(), #Val. Unitário
                #tables[2].find_all('td')[3].text.strip(), #Val. Negociado      
                tables[1].find_all('td')[3].text.strip(), #Qtd. Saldo
                url, #link de cada item no Comprasgov
                ]
            #Append a linha
            linhas.append(linha)
            
    #Para obter os cabeçalhos

    headers = [
        "Número da Compra",
        tables[0].find_all('strong')[0].text.strip(), #Item
        tables[0].find_all('strong')[2].text.strip(), #Descrição
        tables[0].find_all('strong')[3].text.strip(), #Descrição detalhada
        #tables[0].find_all('strong')[4].text.strip(), #Qtd. Total
        tables[0].find_all('strong')[5].text.strip(), #Vig. Início ARP
        tables[0].find_all('strong')[6].text.strip(), #Vig. Fim ARP
        tables[1].find_all('th')[0].text.strip(), #Unidade
        #tables[1].find_all('th')[1].text.strip(), #Tipo UASG
        tables[1].find_all('th')[2].text.strip(), #Qtd. Autorizada

        tables[2].find_all('th')[0].text.strip(), #Fornecedor
        #tables[2].find_all('th')[1].text.strip(), #Qtd. Homologada
        tables[2].find_all('th')[2].text.strip(), #Val. Unitário
        #tables[2].find_all('th')[3].text.strip(), #Val. Negociado
        tables[1].find_all('th')[3].text.strip(), #Qtd. Saldo
        'Link do Item'
    ]

    headers[1] = "Número do Item"
    headers[2] = "Descrição"
    headers[3] = "Descrição Detalhada"
    headers[4] = "Início da Vigência"
    headers[5] = "Fim da Vigência"

    #transformando os dados em um dataframe

    df_itens_novos = pd.DataFrame(linhas, columns = headers)

    #Corrigindo os valores que são numéricos

    df_itens_novos['Qtd. Autorizada'] = df_itens_novos['Qtd. Autorizada'].str.replace('.','')
    df_itens_novos['Qtd. Autorizada'] = df_itens_novos['Qtd. Autorizada'].str.replace(',','.')

    df_itens_novos['Val. Unitário'] = df_itens_novos['Val. Unitário'].str.replace('.','')
    df_itens_novos['Val. Unitário'] = df_itens_novos['Val. Unitário'].str.replace(',','.')

    df_itens_novos['Qtd. Saldo'] = df_itens_novos['Qtd. Saldo'].str.replace('.','')
    df_itens_novos['Qtd. Saldo'] = df_itens_novos['Qtd. Saldo'].str.replace(',','.')

    
    #Inserindo novas colunas

    tipo_compra = []
    numero_compra_pregao = []
    ano_compra_pregao = []


    for i in range(len(df_itens_novos['Número da Compra'])):
        tipo = df_itens_novos['Número da Compra'][i].split(' ')[0]
        numero_pregao = df_itens_novos['Número da Compra'][i].split(' ')[2].split('/')[0]
        ano_pregao = df_itens_novos['Número da Compra'][i].split(' ')[2].split('/')[1]
        

        tipo_compra.append(tipo)
        numero_compra_pregao.append(numero_pregao)
        ano_compra_pregao.append(ano_pregao)
        

    df_itens_novos['Tipo de Compra'] = tipo_compra
    df_itens_novos['Número do Pregão'] = numero_compra_pregao
    df_itens_novos['Ano do Pregão'] = ano_compra_pregao
    df_itens_novos['UASG'] = df_itens_novos['Unidade'].str[:6]

    #corrigindo o tipo
    df_itens_novos['Número do Item'] = df_itens_novos['Número do Item'].astype(int)
    df_itens_novos['Número do Item'] = df_itens_novos['Número do Item'].astype('string')
    
    df_itens_novos['Número do Pregão'] = df_itens_novos['Número do Pregão'].astype(int)
    df_itens_novos['Número do Pregão'] = df_itens_novos['Número do Pregão'].astype('string')
    
    df_itens_novos['Ano do Pregão'] = df_itens_novos['Ano do Pregão'].astype(int)
    df_itens_novos['Ano do Pregão'] = df_itens_novos['Ano do Pregão'].astype('string')

    #apagar
    print('imprimir df_itens_novos')
    display(df_itens_novos)

    #Função para obter a unidade de fornecimento e objeto a partir do site antigo
    
    df_itens_novos_site_antigo = obter_unidade_objeto(df_itens_novos)

    df_itens_novos = merge_df(df_itens_novos,df_itens_novos_site_antigo)

    return df_itens_novos

In [53]:
def merge_df(df_itens_novos,df_itens_novos_site_antigo):
    
    #Criando novas colunas que servirão como key para o merge
    df_itens_novos['key'] = df_itens_novos['Número do Pregão'] + '_' + df_itens_novos['Ano do Pregão'] + '_' + df_itens_novos['Número do Item']
    df_itens_novos_site_antigo['key'] = df_itens_novos_site_antigo['Número do Pregão'] + '_' + df_itens_novos_site_antigo['Ano do Pregão']+ '_' + df_itens_novos_site_antigo['Número do Item']


    #apagar

    display(df_itens_novos['key'])
    display(df_itens_novos_site_antigo['key'])
    
    #obtendo o df final

    
    df_itens_novos = df_itens_novos.merge(df_itens_novos_site_antigo, how = 'left', on = ['key'])


    
    df_itens_novos = df_itens_novos.drop(columns=['key',
                                                    'Número da Compra_y',
                                                  'Número do Item_y',
                                                  'Descrição_y',
                                                  'Descrição Complementar',
                                                  'Ano do Pregão_y', 
                                                  'Número do Pregão_y'])
    
    df_itens_novos.columns = ['Número da Compra', 'Número do Item', 'Descrição',
           'Descrição Detalhada', 'Início da Vigência', 'Fim da Vigência',
           'Unidade', 'Qtd. Autorizada', 'Fornecedor', 'Val. Unitário',
           'Qtd. Saldo', 'Link do Item', 'Tipo de Compra', 'Número do Pregão',
           'Ano do Pregão', 'UASG', 'Objeto','Unidade de Fornecimento',]

    df_itens_novos['Descrição'] = df_itens_novos['Descrição'].str.replace(';',',')
    df_itens_novos['Descrição Detalhada'] = df_itens_novos['Descrição Detalhada'].str.replace(';',',')
    df_itens_novos['Objeto'] = df_itens_novos['Objeto'].str.replace(';',',')
    df_itens_novos['Unidade de Fornecimento'] = df_itens_novos['Unidade de Fornecimento'].str.replace(';',',')

    return df_itens_novos

In [54]:
def atualizar_saldo(df_itens):
    
#atualizando o saldo
    
    for i in range(len(df_itens['Link do Item'])):
        
        url = df_itens['Link do Item'][i]
        
        # Enviar uma solicitação GET para obter o conteúdo da página
        response = requests.get(url)
        
        # Verificar se a solicitação foi bem-sucedida
    
        if response.status_code == 200:  

            # Parsear o HTML da página com Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Encontrar todas as tabelas na página
            tables = soup.find_all('table')

            #Define a nova linha
            saldo = tables[1].find_all('td')[3].text.strip() #Qtd. Saldo

            #Gravar saldo
            df_itens['Qtd. Saldo'][i] = saldo

            print('Saldo do item ' + str(i) + ' atualizado com sucesso!')

    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace('.','')
    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace(',','.')
    
    return df_itens

In [55]:
if __name__ == "__main__":
    iniciar()

Lista de Pregões salvas no PC encontrada!
Baixando nova lista de pregões ...
Nova Lista de pregões salva com sucesso!
Há novos pregões não cadastrados! 
Obtendo os dados dos pregões: 
['https://contratos.sistema.gov.br/transparencia/compras/539777/itens']
https://contratos.sistema.gov.br/transparencia/compras/539777/itens
['https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303503/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303505/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303506/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303509/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303511/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303513/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303515/show', 'https://contratos.sistema.gov.br/transparencia/compras/539777/itens/5303516/show', 'ht

,Número da Compra,Número do Item,Descrição,Descrição Detalhada,Início da Vigência,Fim da Vigência,Unidade,Qtd. Autorizada,Fornecedor,Val. Unitário,Qtd. Saldo,Link do Item,Tipo de Compra,Número do Pregão,Ano do Pregão,UASG
0,Pregão | 90004/2024,1,481558 - ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO MOTORES A DIESEL, TIPO ...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,30.00000,14.974.969/0001-78 - ROGAMA DISTRIBUIDORA E SE...,550.00,30.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
1,Pregão | 90004/2024,2,472896 - ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO AUTOMOTIVO, TIPO MINERA...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,20.00000,52.743.219/0001-84 - GSW DISTRIBUIDORA LTDA,419.00,20.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
2,Pregão | 90004/2024,3,471985 - ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,10.00000,52.743.219/0001-84 - GSW DISTRIBUIDORA LTDA,615.00,10.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
3,Pregão | 90004/2024,4,471985 - ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,30.00000,52.743.219/0001-84 - GSW DISTRIBUIDORA LTDA,509.00,30.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
4,Pregão | 90004/2024,5,463190 - ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,50.00000,14.974.969/0001-78 - ROGAMA DISTRIBUIDORA E SE...,21.90,50.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
5,Pregão | 90004/2024,6,601273 - ADITIVO RADIADOR,"ADITIVO RADIADOR, CARACTERÍSTICAS ADICIONAIS N...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,30.00000,51.560.442/0001-23 - DUFILTER DISTRIBUIDORA DE...,154.95,30.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
6,Pregão | 90004/2024,7,478331 - QUEROSENE,"QUEROSENE, USO LIMPEZA, DESENGRAXANTE, SOLVENT...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,600.00000,13.991.459/0001-46 - BAZA DISTRIBUIDORA LTDA,19.15,600.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
7,Pregão | 90004/2024,8,251546 - FLUIDO HIDRÁULICO,"FLUIDO HIDRÁULICO, ORIGEM MINERAL, VISCOSIDADE...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,150.00000,08.687.885/0001-50 - X5 COMERCIO DE PRODUTOS A...,401.81,150.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
8,Pregão | 90004/2024,9,617101 - ADITIVO,"ADITIVO, COMPONENTE BÁSICO ANTICORROSIVO E FUN...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,20.00000,13.991.459/0001-46 - BAZA DISTRIBUIDORA LTDA,889.99,20.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224
9,Pregão | 90004/2024,10,349296 - ANTICORROSIVO,"ANTICORROSIVO, APRESENTAÇÃO SPRAY, ASPECTO FÍS...",04/07/2024,04/07/2025,160224 - PQ R MNT/5,200.00000,04.341.779/0001-60 - SANEX COMERCIO E SERVICOS...,7.00,200.00000,https://contratos.sistema.gov.br/transparencia...,Pregão,90004,2024,160224


imprimir df_itens_novos_site_antigo


,Número da Compra,Objeto,Número do Item,Descrição,Descrição Complementar,Unidade de Fornecimento
0,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,1,ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO MOTORES A DIESEL, TIPO ...","Galão 20,00 L"
1,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,2,ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO AUTOMOTIVO, TIPO MINERA...","Galão 20,00 L"
2,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,3,ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...","Galão 20,00 L"
3,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,4,ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...","Galão 20,00 L"
4,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,5,ÓLEO LUBRIFICANTE,"ÓLEO LUBRIFICANTE, USO INDUSTRIAL, TIPO MINERA...",Litro
5,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,6,ADITIVO RADIADOR,"ADITIVO RADIADOR, CARACTERÍSTICAS ADICIONAIS N...","Galão 20,00 L"
6,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,7,QUEROSENE,"QUEROSENE, USO LIMPEZA, DESENGRAXANTE, SOLVENT...",Litro
7,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,8,FLUIDO HIDRÁULICO,"FLUIDO HIDRÁULICO, ORIGEM MINERAL, VISCOSIDADE...",Unidade
8,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,9,ADITIVO,"ADITIVO, COMPONENTE BÁSICO ANTICORROSIVO E FUN...",Unidade
9,90004/2024,Objeto: Objeto: Pregão Eletrônico - Aquisição ...,10,ANTICORROSIVO,"ANTICORROSIVO, APRESENTAÇÃO SPRAY, ASPECTO FÍS...",Unidade


0      90004_2024_1
1      90004_2024_2
2      90004_2024_3
3      90004_2024_4
4      90004_2024_5
5      90004_2024_6
6      90004_2024_7
7      90004_2024_8
8      90004_2024_9
9     90004_2024_10
10    90004_2024_11
11    90004_2024_12
12    90004_2024_13
13    90004_2024_14
14    90004_2024_15
15    90004_2024_16
16    90004_2024_17
17    90004_2024_18
18    90004_2024_19
19    90004_2024_20
20    90004_2024_21
21    90004_2024_22
22    90004_2024_23
23    90004_2024_24
Name: key, dtype: string

0      90004_2024_1
1      90004_2024_2
2      90004_2024_3
3      90004_2024_4
4      90004_2024_5
5      90004_2024_6
6      90004_2024_7
7      90004_2024_8
8      90004_2024_9
9     90004_2024_10
10    90004_2024_11
11    90004_2024_12
12    90004_2024_13
13    90004_2024_14
14    90004_2024_15
15    90004_2024_16
16    90004_2024_17
17    90004_2024_18
18    90004_2024_19
19    90004_2024_20
20    90004_2024_21
21    90004_2024_22
22    90004_2024_23
23    90004_2024_24
Name: key, dtype: string

Dados dos novos pregões atualizados!
Tentando carregar dados dos pregões anteriores...
Dados carregados com sucesso! Iniciando atualização dos saldos...


C:\Users\PQRMNT 5\AppData\Local\Temp\ipykernel_16072\1924332210.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_itens['Qtd. Saldo'][i] = saldo
C:\Users\PQRMNT 5\AppData\Local\Temp\ipykernel_16072\1924332210.py:26: SettingWithCopyWarn

Saldo do item 0 atualizado com sucesso!
Saldo do item 1 atualizado com sucesso!
Saldo do item 2 atualizado com sucesso!
Saldo do item 3 atualizado com sucesso!
Saldo do item 4 atualizado com sucesso!
Saldo do item 5 atualizado com sucesso!
Saldo do item 6 atualizado com sucesso!
Saldo do item 7 atualizado com sucesso!
Saldo do item 8 atualizado com sucesso!
Saldo do item 9 atualizado com sucesso!
Saldo do item 10 atualizado com sucesso!
Saldo do item 11 atualizado com sucesso!
Saldo do item 12 atualizado com sucesso!
Saldo do item 13 atualizado com sucesso!
Saldo do item 14 atualizado com sucesso!
Saldo do item 15 atualizado com sucesso!
Saldo do item 16 atualizado com sucesso!
Saldo do item 17 atualizado com sucesso!
Saldo do item 18 atualizado com sucesso!
Saldo do item 19 atualizado com sucesso!
Saldo do item 20 atualizado com sucesso!
Saldo do item 21 atualizado com sucesso!
Saldo do item 22 atualizado com sucesso!
Saldo do item 23 atualizado com sucesso!
Saldo do item 24 atualizad